## [Day 1: Sonar Sweep](https://adventofcode.com/2021/day/1)

In [1]:
using DelimitedFiles

In [2]:
depths = vec(readdlm("1.txt", '\n', Int))
sum(diff(depths) .> 0)

1709

In [3]:
sum(diff(sum(depths[1+i:end-2+i] for i in 0:2)) .> 0)

1761

## [Day 2: Dive!](https://adventofcode.com/2021/day/2)

In [4]:
using DelimitedFiles

In [5]:
commands = readdlm("2.txt")
forward_indices = commands[:, 1] .== "forward"
up_indices = commands[:, 1] .== "up"
down_indices = commands[:, 1] .== "down"
sum(commands[forward_indices, 2]) * (sum(commands[down_indices, 2]) - sum(commands[up_indices, 2]))

1580000

In [6]:
position = depth = aim = 0
for i in 1:size(commands, 1)
    x = commands[i, 2]
    if commands[i] == "up"
        aim -= x
    elseif commands[i] == "down"
        aim += x
    else
        position += x
        depth += aim * x
    end
end
position * depth

1251263225

## [Day 3: Binary Diagnostic](https://adventofcode.com/2021/day/3)

In [7]:
bits = parse.(Bool, hcat(collect.(readlines("3.txt"))...))
gamma_digits = vec(sum(bits, dims = 2)) .> size(bits, 2) / 2
beta_digits = 1 .- gamma_digits
binary_digits_to_int(v) = sum(2^(i - 1) * d for (i, d) in enumerate(reverse(v)))

binary_digits_to_int(gamma_digits) * binary_digits_to_int(beta_digits)

1092896

In [8]:
function rating(bits, oxygen = true)
    for i in 1:size(bits, 1)
        if oxygen == (sum(bits[i, :]) >= size(bits, 2) / 2)
            bits = bits[:, bits[i, :]]
        else
            bits = bits[:, .!bits[i, :]]
        end
        if size(bits, 2) == 1
            return binary_digits_to_int(vec(bits))
        end
    end
end

rating(bits) * rating(bits, false)

4672151

## [Day 4: Giant Squid](https://adventofcode.com/2021/day/4)

In [9]:
using DelimitedFiles

In [10]:
input_4 = readdlm("4.txt")
numbers = parse.(Int, split(input_4[1], ','))
boards = permutedims(reshape(Int.(input_4[2:end, :]), 5, :, 5), (1, 3, 2))
is_winning(board) = any(all(board .< 0, dims = 1)) || any(all(board .< 0, dims = 2))

function bingo(numbers, boards, first_or_last = :first)
    boards = copy(boards)
    chosen_board = nothing
    for (round, i) in enumerate(numbers)
        boards[boards.==i] .= -round
        winners = [is_winning(boards[:, :, j]) for j in 1:size(boards, 3)]
        if first_or_last === :first
            winner = findfirst(winners)
            if winner !== nothing
                chosen_board = boards[:, :, winner]
                return i * sum(chosen_board[chosen_board.>=0])
            end
        else
            if length(winners) == 1 && only(winners)
                return i * sum(boards[boards.>=0])
            else
                boards = boards[:, :, .!winners]
            end
        end
    end
end

bingo(numbers, boards, :first), bingo(numbers, boards, :last)

(25023, 2634)

## [Day 5: Hydrothermal Venture](https://adventofcode.com/2021/day/5)

In [11]:
lines =
    reshape(
        hcat([parse.(Int, line[[1, 2, 4, 5]]) for line in split.(readlines("5.txt"), ((',', ' '),))]...),
        (2, 2, :),
    ) .+ 1

function count_overlaps(lines, include_diagonals = false)
    vents = zeros(Int, maximum(lines[1, :, :]), maximum(lines[2, :, :]))
    for i in 1:size(lines, 3)
        x1, y1, x2, y2 = lines[:, :, i]
        if x1 == x2
            vents[x1, range(minmax(y1, y2)...)] .+= 1
        elseif y1 == y2
            vents[range(minmax(x1, x2)...), y2] .+= 1
        elseif include_diagonals && abs(x1 - x2) == abs(y1 - y2)
            for (x, y) in zip(x1:(x1 <= x2 ? 1 : -1):x2, y1:(y1 <= y2 ? 1 : -1):y2)
                vents[x, y] += 1
            end
        end
    end
    sum(vents .>= 2)
end

count_overlaps(lines), count_overlaps(lines, true)

(7644, 18627)

## [Day 6: Lanternfish](https://adventofcode.com/2021/day/6)

In [12]:
using StatsBase

In [13]:
fish = parse.(Int, split(readline("6.txt"), ','))
population = counts(fish, 0:8)

function evolve_population(population, steps)
    population = copy(population)
    for t in 1:steps
        new_fish = population[1]
        population[1:8] .= population[2:9]
        population[7] += new_fish
        population[9] = new_fish
    end
    population
end

sum(evolve_population(population, 80)), sum(evolve_population(population, 256))

(345793, 1572643095893)

## [Day 7: The Treachery of Whales](https://adventofcode.com/2021/day/7)

In [14]:
crabs = parse.(Int, split(readline("7.txt"), ','))
sum(abs.(crabs .- median(crabs)))

356958.0

In [15]:
function minimize_quadratic_crab_fuel(crabs)
    num_crabs = length(crabs)
    x = m = mean(crabs)
    while true
        new_x = m - sum(sign.(x .- crabs)) / (2 * num_crabs)
        new_x == x && break
        x = new_x
    end
    total_fuel(x) = sum((y -> y * (y + 1) ÷ 2).(abs.(crabs .- floor(x))))
    min(total_fuel.((floor(Int, x), ceil(Int, x)))...)
end

minimize_quadratic_crab_fuel(crabs)

105461913

## [Day 8: Seven Segment Search](https://adventofcode.com/2021/day/8)

In [16]:
using DelimitedFiles

In [17]:
input_8 = permutedims(readdlm("8.txt"))
all_digits = Bool[
    1 0 1 1 0 1 1 1 1 1
    1 0 0 0 1 1 1 0 1 1
    1 1 1 1 1 0 0 1 1 1
    0 0 1 1 1 1 1 0 1 1
    1 0 1 0 0 0 1 0 1 0
    1 1 0 1 1 1 1 1 1 1
    1 0 1 1 0 1 1 0 1 1
]
@show sum(all_digits, dims = 1)
@show sum(all_digits, dims = 2)
digits_patterns = input_8[1:10, :]
output_patterns = input_8[end-3:end, :]

function patterns_to_vectors(patterns)
    vectors = zeros(Bool, 7, size(patterns)...)
    for i in CartesianIndices(patterns)
        for c in patterns[i]
            vectors[Int(c)-96, i] = true
        end
    end
    vectors
end

digits_vectors = patterns_to_vectors(digits_patterns)
output_vectors = patterns_to_vectors(output_patterns)

sum(sum(sum(output_vectors, dims = 1) .== n) for n in sum(all_digits[:, [1, 4, 7, 8].+1], dims = 1))

sum(all_digits, dims = 1) = [6 2 5 5 4 5 6 3 7 6]
sum(all_digits, dims = 2) = [8; 6; 8; 7; 4; 9; 7;;]


539

In [18]:
function decode(digits_vectors)
    n_by_digit = vec(sum(digits_vectors, dims = 1))
    n_by_segment = vec(sum(digits_vectors, dims = 2))

    segments = zeros(Int, 7)
    segments[2] = findfirst(n_by_segment .== 6)
    segments[5] = findfirst(n_by_segment .== 4)
    segments[6] = findfirst(n_by_segment .== 9)
    d1 = findfirst(n_by_digit .== 2)
    d4 = findfirst(n_by_digit .== 4)
    d7 = findfirst(n_by_digit .== 3)
    segments[1] = findfirst(digits_vectors[:, d1] .⊻ digits_vectors[:, d7])
    sc = findfirst(digits_vectors[:, d1])
    segments[3] = sc == segments[6] ? findnext(digits_vectors[:, d1], sc + 1) : sc
    b_and_d = digits_vectors[:, d1] .⊻ digits_vectors[:, d4]
    sd = findfirst(b_and_d)
    segments[4] = any(segments .== sd) ? findnext(b_and_d, sd + 1) : sd
    segments[7] = 28 - sum(segments)
    segments
end


vector_to_digit_map = Dict(all_digits[:, i] => i - 1 for i in 1:10)
total = 0
for i in 1:size(output_vectors, 3)
    output = output_vectors[decode(digits_vectors[:, :, i]), :, i]
    total += sum(vector_to_digit_map[output[:, i]] * 10^(4 - i) for i in 1:4)
end
total

1084606

## [Day 9: Smoke Basin](https://adventofcode.com/2021/day/9)

In [19]:
heightmap = parse.(Int, hcat(collect.(readlines("9.txt"))...))
grid_neighbors = (CartesianIndex(1, 0), CartesianIndex(-1, 0), CartesianIndex(0, 1), CartesianIndex(0, -1))

total = 0
for I in CartesianIndices(heightmap)
    level = heightmap[I]
    if all(level < heightmap[I+II] for II in grid_neighbors if I + II in CartesianIndices(heightmap))
        total += level + 1
    end
end
total

572

In [20]:
function compute_basin_sizes(heightmap)
    heightmap = copy(heightmap)
    basin_sizes = Int[]
    while ((I = findfirst(heightmap .< 9)) !== nothing)
        queue = [I]
        count = 0
        while !isempty(queue)
            I = pop!(queue)
            heightmap[I] == 9 && continue
            heightmap[I] = 9
            count += 1
            for II in grid_neighbors
                if I + II in CartesianIndices(heightmap)
                    push!(queue, I + II)
                end
            end
        end
        push!(basin_sizes, count)
    end
    sort(basin_sizes)
end

prod(compute_basin_sizes(heightmap)[end-2:end])

847044

## [Day 10: Syntax Scoring](https://adventofcode.com/2021/day/10)

In [21]:
lines = readlines("10.txt")

lr_map = Dict('(' => ')', '[' => ']', '{' => '}', '<' => '>')
function find_first_illegal_character(line)
    stack = Char[]
    for c in line
        if c in keys(lr_map)
            push!(stack, lr_map[c])
        else
            r = pop!(stack)
            c != r && return c
        end
    end
end

score = Dict(')' => 3, ']' => 57, '}' => 1197, '>' => 25137)
sum(lines) do l
    c = find_first_illegal_character(l)
    c === nothing ? 0 : score[c]
end

345441

In [22]:
function completion_score(line)
    r_score = Dict(')' => 1, ']' => 2, '}' => 3, '>' => 4)
    stack = Char[]
    for c in line
        if c in keys(lr_map)
            push!(stack, lr_map[c])
        else
            r = pop!(stack)
            c != r && return 0
        end
    end

    score = 0
    while !isempty(stack)
        score = 5 * score + r_score[pop!(stack)]
    end
    score
end

Int(median(filter(!iszero, [completion_score(l) for l in lines])))

3235371166

## [Day 11: Dumbo Octopus](https://adventofcode.com/2021/day/11)

In [23]:
octopi = parse.(Int, hcat(collect.(readlines("11.txt"))...))

function octopus_step!(octopi)
    octopi .+= 1
    keep_checking = true
    while keep_checking
        keep_checking = false
        for I in CartesianIndices(octopi)
            if octopi[I] > 9
                octopi[I] = 0
                for N in CartesianIndices((-1:1, -1:1))
                    if I + N in CartesianIndices(octopi) && octopi[I+N] > 0
                        octopi[I+N] += 1
                    end
                end
                keep_checking = true
            end
        end
    end
    octopi
end

function total_flashes(octopi, steps)
    octopi = copy(octopi)
    total = 0
    for _ in 1:100
        total += sum(octopus_step!(octopi) .== 0)
    end
    total
end

total_flashes(octopi, 100)

1661

In [24]:
function first_simultaneous_flash(octopi)
    octopi = copy(octopi)
    for t in 1:1000
        octopus_step!(octopi)
        all(octopi .== 0) && return t
    end
end

first_simultaneous_flash(octopi)

334

## [Day 12: Passage Pathing](https://adventofcode.com/2021/day/12)

In [25]:
edges = split.(readlines("12.txt"), '-')

function cave_paths(edges, allow_single_small_cave_twice = false)
    all_caves = unique(vcat(edges...))
    small_caves = Dict(reverse.(enumerate(filter(x -> islowercase(x[1]), all_caves))))
    large_caves = Dict(reverse.(enumerate(filter(x -> isuppercase(x[1]), all_caves))))
    start_index, end_index = small_caves["start"], small_caves["end"]

    small_large = zeros(Int, length(small_caves), length(large_caves))
    small_small = zeros(Int, length(small_caves), length(small_caves))
    for e in edges
        a, b = minmax(e...)
        if isuppercase(a[1])
            small_large[small_caves[b], large_caves[a]] += 1
        else
            small_small[small_caves[a], small_caves[b]] += 1
            small_small[small_caves[b], small_caves[a]] += 1
        end
    end
    small_small += small_large * small_large'

    total = 0
    paths = [(1, false, [start_index])]
    while !isempty(paths)
        n, small_cave_twice, path = pop!(paths)
        v = path[end]
        for w in 1:length(small_caves)
            w_in_path = w in path
            if w != start_index &&
               small_small[v, w] > 0 &&
               (!w_in_path || (allow_single_small_cave_twice && !small_cave_twice))
                if w == end_index
                    path, n * small_small[v, w]
                    total += n * small_small[v, w]
                else
                    push!(paths, (n * small_small[v, w], small_cave_twice || w_in_path, [path; [w]]))
                end
            end
        end
    end
    total
end

cave_paths(edges), cave_paths(edges, true)

(3421, 84870)

## [Day 13: Transparent Origami](https://adventofcode.com/2021/day/13)

In [26]:
using SparseArrays

In [27]:
input_13 = readlines("13.txt")
blank_line_index = findfirst(==(""), input_13)
points = (x -> tuple(parse.(Int, x)...)).(split.(input_13[1:blank_line_index-1], ','))
folds = input_13[blank_line_index+1:end]

fold_along_x(points, x) = unique([(a < x ? a : 2 * x - a, b) for (a, b) in points])
fold_along_y(points, y) = unique([(a, b < y ? b : 2 * y - b) for (a, b) in points])

function apply_folds(points, folds)
    for f in folds
        if startswith(f, "fold along x=")
            points = fold_along_x(points, parse(Int, f[14:end]))
        else
            points = fold_along_y(points, parse(Int, f[14:end]))
        end
        @show length(points)
    end

    sparse(last.(points) .+ 1, first.(points) .+ 1, 1)
end

apply_folds(points, folds)

length(points) = 693
length(points) = 574
length(points) = 476
length(points) = 397
length(points) = 332
length(points) = 282
length(points) = 242
length(points) = 206
length(points) = 176
length(points) = 147
length(points) = 121
length(points) = 95


6×39 SparseMatrixCSC{Int64, Int64} with 95 stored entries:
⡇⢸⢰⠉⠂⡇⠀⠈⡩⠃⣏⡱⢰⣉⡆⢉⠝⢸⠀⡇
⠑⠊⠈⠒⠁⠓⠒⠘⠒⠂⠃⠑⠘⠀⠃⠓⠒⠈⠒⠁

## [Day 14: Extended Polymerization](https://adventofcode.com/2021/day/14)

In [28]:
using StatsBase

In [29]:
input_14 = readlines("14.txt")
template = input_14[1]
rules = Dict(l[1:2] => (l[1] * l[7], l[7] * l[2]) for l in input_14[3:end])

function apply_rules(pair_counts, rules)
    foldl(mergewith!(+), (Dict(rules[p][i] => c) for (p, c) in pair_counts for i in 1:2); init = typeof(pair_counts)())
end

function character_count_difference(template, rules, steps = 10)
    pair_counts = Dict{String,Int}(countmap(template[i:i+1] for i in 1:length(template)-1))
    pair_counts = foldl((pc, i) -> apply_rules(pc, rules), 1:steps; init = pair_counts)
    char_counts =
        foldl(mergewith!(+), (Dict(p[i] => c) for (p, c) in pair_counts for i in 1:2); init = Dict{Char,Int}())
    char_counts[template[1]] += 1
    char_counts[template[end]] += 1
    map!(x -> x ÷ 2, values(char_counts))
    maximum(values(char_counts)) - minimum(values(char_counts))
end

character_count_difference(template, rules), character_count_difference(template, rules, 40)

(3118, 4332887448171)

## [Day 15: Chiton](https://adventofcode.com/2021/day/15)

In [30]:
using DataStructures

In [31]:
grid = parse.(Int, hcat(collect.(readlines("15.txt"))...))

function min_risk_path(grid)
    neighbors = CartesianIndex.(((1, 0), (-1, 0), (0, 1), (0, -1)))
    cost_to_come = fill(typemax(eltype(grid)), size(grid))
    cost_to_come[1, 1] = 0
    queue = PriorityQueue(CartesianIndex(1, 1) => 0)
    while (v = dequeue!(queue)) != last(CartesianIndices(grid))
        for n in neighbors
            if v + n in CartesianIndices(grid) && cost_to_come[v] + grid[v+n] < cost_to_come[v+n]
                cost_to_come[v+n] = cost_to_come[v] + grid[v+n]
                queue[v+n] = cost_to_come[v+n]
            end
        end
    end
    cost_to_come[end, end]
end

min_risk_path(grid), min_risk_path(hvcat(5, [mod.(grid .+ i .+ j, (1:9,)) for i in 0:4, j in 0:4]...))

(390, 2814)

## [Day 16: Packet Decoder](https://adventofcode.com/2021/day/16)

In [32]:
input_16 = readline("16.txt")
bits = vec(permutedims(hcat([(1 << i) .& hex2bytes(input_16) .!= 0 for i in 7:-1:0]...)))

bits_to_int(bits) = sum((2 .^ (length(bits)-1:-1:0))[bits])

function parse_packet(bits, sum_version_numbers = true)
    reduction_map = Dict(
        0 => sum,
        1 => prod,
        2 => minimum,
        3 => maximum,
        5 => x -> Int(x[1] > x[2]),
        6 => x -> Int(x[1] < x[2]),
        7 => x -> Int(x[1] == x[2]),
    )

    version = bits_to_int(bits[1:3])
    type_id = bits_to_int(bits[4:6])
    if type_id == 4
        literal = 0
        local i
        for outer i in 7:5:length(bits)
            literal <<= 4
            literal += bits_to_int(bits[i+1:i+4])
            !bits[i] && break
        end
        return sum_version_numbers ? version : literal, bits[i+5:end]
    end

    reduction = reduction_map[type_id]
    length_type = bits[7]
    if length_type
        num_sub_packets = bits_to_int(bits[8:18])
        bits = bits[19:end]
        values = Int[]
        for _ in 1:num_sub_packets
            v, bits = parse_packet(bits, sum_version_numbers)
            push!(values, v)
        end
        return sum_version_numbers ? version + sum(values) : reduction(values), bits
    else
        bit_length_sub_packets = bits_to_int(bits[8:22])
        bits = bits[23:end]
        values = Int[]
        starting_num_bits = length(bits)
        while starting_num_bits - length(bits) < bit_length_sub_packets
            v, bits = parse_packet(bits, sum_version_numbers)
            push!(values, v)
        end
        return sum_version_numbers ? version + sum(values) : reduction(values), bits
    end
end

parse_packet(bits), parse_packet(bits, false)

((981, Bool[0, 0, 0, 0, 0, 0]), (299227024091, Bool[0, 0, 0, 0, 0, 0]))

## [Day 17: Trick Shot](https://adventofcode.com/2021/day/17)

In [33]:
@show input_17 = readline("17.txt")
x_range, y_range =
    (r -> parse(Int, r[1]):parse(Int, r[2])).(split.(split(input_17[findfirst('=', input_17)+1:end], ", y="), ".."))

sum(1:abs(y_range.start)-1)

input_17 = readline("17.txt") = "target area: x=179..201, y=-109..-63"


5886

In [34]:
function count_feasible_velocity_values(x_range, y_range)
    n = 0
    for x_vel in 1:x_range.stop
        for y_vel in y_range.start:abs(y_range.start)-1
            x = y = 0
            xv, yv = x_vel, y_vel
            while y >= y_range.start
                x += xv
                y += yv
                xv = max(xv - 1, 0)
                yv -= 1
                if x in x_range && y in y_range
                    n += 1
                    break
                end
            end
        end
    end
    n
end

count_feasible_velocity_values(x_range, y_range)

1806

## [Day 18: Snailfish](https://adventofcode.com/2021/day/18)

In [35]:
struct SnailfishNumber
    l::Vector{Tuple{Int,Int}}

    function SnailfishNumber(l)
        l = copy(l)
        while true
            e = findfirst(x -> x[1] > 4, l)
            if e !== nothing
                d, v1 = l[e]
                _, v2 = l[e+1]
                if e - 1 in 1:length(l)
                    l[e-1] = (l[e-1][1], l[e-1][2] + v1)
                end
                if e + 2 in 1:length(l)
                    l[e+2] = (l[e+2][1], l[e+2][2] + v2)
                end
                deleteat!(l, e)
                l[e] = (d - 1, 0)
                continue
            end
            s = findfirst(x -> x[2] >= 10, l)
            if s !== nothing
                d, v = l[s]
                l[s] = (d + 1, ceil(Int, v / 2))
                insert!(l, s, (d + 1, floor(Int, v / 2)))
                continue
            end
            break
        end
        new(l)
    end
end

function SnailfishNumber(s::String)
    l = Tuple{Int,Int}[]
    d = 0
    for c in s
        if c == '['
            d += 1
        elseif c == ']'
            d -= 1
        elseif c != ','
            push!(l, (d, parse(Int, c)))
        end
    end
    SnailfishNumber(l)
end

function Base.:+(s1::SnailfishNumber, s2::SnailfishNumber)
    SnailfishNumber([[(d + 1, v) for (d, v) in s1.l]; [(d + 1, v) for (d, v) in s2.l]])
end

function magnitude(s::SnailfishNumber)
    l = copy(s.l)
    while length(l) > 1
        i = argmax(first.(l))
        d, v1 = l[i]
        _, v2 = l[i+1]
        deleteat!(l, i)
        l[i] = (d - 1, 3 * v1 + 2 * v2)
    end
    only(l)[2]
end

snailfish_numbers = [SnailfishNumber(s) for s in readlines("18.txt")]

(
    magnitude(foldl(+, snailfish_numbers)),
    maximum(
        max(
            magnitude(snailfish_numbers[i] + snailfish_numbers[j]),
            magnitude(snailfish_numbers[j] + snailfish_numbers[i]),
        ) for i in 1:length(snailfish_numbers) for j in i+1:length(snailfish_numbers)
    ),
)

(3411, 4680)

## [Day 19: Beacon Scanner](https://adventofcode.com/2021/day/19)

In [36]:
using LinearAlgebra
using StaticArrays

In [37]:
input_19 = push!(readlines("19.txt"), "")
scanners = [
    (x -> SVector{3}(parse.(Int, split(x, ',')))).(input_19[s+1:e-1]) for
    (s, e) in zip(findall(contains("scanner"), input_19), findall(==(""), input_19))
]

rotations = filter!(
    x -> det(x) == 1,
    [
        SMatrix{3,3}(I)[:, SVector(permutation)] .* (-1) .^ SVector(Tuple(signs)) for
        permutation in ((1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)) for
        signs in CartesianIndices((2, 2, 2))
    ],
)

function align_scanner(reference_scanner, scanner, rotations)
    reference_set = Set(reference_scanner)
    for R in rotations
        rotated_scanner = (R,) .* scanner
        for r in reference_scanner[12:end]
            for o in rotated_scanner
                translation = r - o
                aligned_scanner = rotated_scanner .+ (translation,)
                if count(in(reference_set), aligned_scanner) >= 12
                    return aligned_scanner, translation
                end
            end
        end
    end
end

function align_scanners(scanners, rotations)
    aligned_scanners = [(scanners[1], zeros(SVector{3,Int}))]
    unaligned_scanners = scanners[2:end]
    for i in 1:length(scanners)
        a, _ = aligned_scanners[i]
        still_unaligned_scanners = eltype(unaligned_scanners)[]
        for u in unaligned_scanners
            if (aligned_scanner = align_scanner(a, u, rotations)) !== nothing
                push!(aligned_scanners, aligned_scanner)
            else
                push!(still_unaligned_scanners, u)
            end
        end
        unaligned_scanners = still_unaligned_scanners
    end
    aligned_scanners
end

aligned_scanners = align_scanners(scanners, rotations)
beacon_locations = unique(vcat(first.(aligned_scanners)...))
scanner_locations = last.(aligned_scanners)
length(beacon_locations), maximum(sum(abs.(a .- b)) for a in scanner_locations for b in scanner_locations)

(342, 9668)

## [Day 20: Trench Map](https://adventofcode.com/2021/day/20)

In [38]:
input_20 = readlines("20.txt")
output_map = permutedims(reshape([c == '#' for c in input_20[1]], [2 for _ in 1:9]...), 9:-1:1)
image = hcat((x -> [c == '#' for c in x]).(input_20[3:end])...)

function enhance(image, output_map, pad = falses)
    padded_image = pad(size(image) .+ 4)
    padded_image[3:end-2, 3:end-2] = image
    [
        output_map[Tuple(padded_image[P+I-CartesianIndex(1, 1)] + 1 for I in CartesianIndices((3, 3)))...] for
        P in CartesianIndices(size(image) .+ 2)
    ]
end

sum(enhance(enhance(image, output_map), output_map, trues)),
sum(foldl((image, i) -> enhance(image, output_map, isodd(i) ? falses : trues), 1:50; init = image))

(5425, 14052)

## [Day 21: Dirac Dice](https://adventofcode.com/2021/day/21)

In [39]:
using StatsBase

In [40]:
p1, p2 = (l -> parse.(Int, l[end])).(readlines("21.txt"))

function deterministic_dice(p1, p2)
    s1 = s2 = 0
    for r in 1:Int(1e9)
        m = mod(r, 1:6)
        if m <= 3
            p1 += r
            if m == 3
                p1 = mod(p1, 1:10)
                s1 += p1
                s1 >= 1000 && return s2 * r
            end
        else
            p2 += r
            if m == 6
                p2 = mod(p2, 1:10)
                s2 += p2
                s2 >= 1000 && return s1 * r
            end
        end
    end
end

function dirac_dice(p1, p2)
    outcomes = counts([sum(Tuple(I)) for I in CartesianIndices((3, 3, 3))])
    live_universes = Dict((true, p1, p2, 0, 0) => 1)
    done_universes = typeof(live_universes)()
    while !isempty(live_universes)
        universe, count = pop!(live_universes)
        p, p1, p2, s1, s2 = universe
        for (i, c) in enumerate(outcomes)
            if p
                new_p1 = mod(p1 + i + 2, 1:10)
                new_s1 = s1 + new_p1
                new_universe = Dict((!p, new_p1, p2, new_s1, s2) => count * c)
                mergewith!(+, new_s1 >= 21 ? done_universes : live_universes, new_universe)
            else
                new_p2 = mod(p2 + i + 2, 1:10)
                new_s2 = s2 + new_p2
                new_universe = Dict((!p, p1, new_p2, s1, new_s2) => count * c)
                mergewith!(+, new_s2 >= 21 ? done_universes : live_universes, new_universe)
            end
        end
    end

    max(sum(c for (k, c) in done_universes if k[1]), sum(c for (k, c) in done_universes if !k[1]))
end

deterministic_dice(p1, p2), dirac_dice(p1, p2)

(1002474, 919758187195363)

## [Day 22: Reactor Reboot](https://adventofcode.com/2021/day/22)

In [41]:
input_22 = readlines("22.txt")
instructions = (x -> (x[1] == "on", (s -> parse.(Int, split(s[3:end], ".."))).(split(x[2], ',')))).(split.(input_22))

function count_on_cells(instructions)
    bounds = reduce(hcat, reduce.(vcat, last.(instructions)))
    bounds = [bounds (bounds .+ 1)]
    x_bounds = sort(unique(bounds[1:2, :]))
    y_bounds = sort(unique(bounds[3:4, :]))
    z_bounds = sort(unique(bounds[5:6, :]))
    x_to_i = Dict(x => i for (i, x) in enumerate(x_bounds))
    y_to_i = Dict(y => i for (i, y) in enumerate(y_bounds))
    z_to_i = Dict(z => i for (i, z) in enumerate(z_bounds))

    A = falses(length(x_bounds), length(y_bounds), length(z_bounds))
    for (o, c) in instructions
        x, y, z = c
        A[x_to_i[x[1]]:x_to_i[x[2]], y_to_i[y[1]]:y_to_i[y[2]], z_to_i[z[1]]:z_to_i[z[2]]] .= o
    end
    dx, dy, dz = diff.((x_bounds, y_bounds, z_bounds))
    sum(dx[I[1]] * dy[I[2]] * dz[I[3]] for I in CartesianIndices(A) if A[I])
end

count_on_cells(instructions[1:findfirst(x -> !(x[2][1][1] in -50:50), instructions)-1]), count_on_cells(instructions)

(583641, 1182153534186233)

## [Day 23: Amphipod](https://adventofcode.com/2021/day/23)

In [42]:
import Base.setindex
using DataStructures

In [43]:
input_23 = readlines("23.txt")
rooms = tuple((r .- 'A' for r in zip(input_23[3][4:2:10], input_23[4][4:2:10]))...)

make_state(rooms) = (ntuple(_ -> -1, Val(11)), rooms)

function neighbors(state)
    stopping_spots = (1, 2, 4, 6, 8, 10, 11)
    hallway, rooms = state
    room_size = length(rooms[1])
    neighbors = typeof((state, 0))[]
    for s in stopping_spots
        v = hallway[s]
        if v >= 0
            r = v + 1
            t = 2 * r + 1
            if all(h == s || hallway[h] < 0 for h in range(minmax(s, t)...))
                for d in 1:room_size
                    if rooms[r] == ntuple(i -> i <= d ? -1 : v, room_size)
                        push!(
                            neighbors,
                            (
                                (setindex(hallway, -1, s), setindex(rooms, setindex(rooms[r], v, d), r)),
                                (abs(s - t) + d) * 10^v,
                            ),
                        )
                        break
                    end
                end
            end
        end
    end
    for (r, room) in enumerate(rooms)
        t = 2 * r + 1
        for (d, v) in enumerate(room)
            if v >= 0
                for s in stopping_spots
                    if all(hallway[h] < 0 for h in range(minmax(s, t)...))
                        push!(
                            neighbors,
                            (
                                (setindex(hallway, v, s), setindex(rooms, setindex(rooms[r], -1, d), r)),
                                (abs(s - t) + d) * 10^v,
                            ),
                        )
                    end
                end
                break
            end
        end
    end
    neighbors
end

function organize(rooms)
    start_state = make_state(rooms)
    final_state = make_state(ntuple(i -> (i - 1) .+ zero.(rooms[1]), 4))

    neighbor_dict = DefaultDict{typeof(start_state),Vector{typeof((start_state, 0))}}(neighbors; passkey = true)
    cost_to_come = DefaultDict{typeof(start_state),Int}(typemax(Int))
    cost_to_come[start_state] = 0
    queue = PriorityQueue(start_state => 0)
    while (s = dequeue!(queue)) != final_state
        c = cost_to_come[s]
        for (n, d) in neighbor_dict[s]
            if c + d < cost_to_come[n]
                cost_to_come[n] = c + d
                queue[n] = cost_to_come[n]
            end
        end
    end
    cost_to_come[final_state]
end

organize(rooms),
organize((
    (rooms[1][1], 3, 3, rooms[1][2]),
    (rooms[2][1], 2, 1, rooms[2][2]),
    (rooms[3][1], 1, 0, rooms[3][2]),
    (rooms[4][1], 0, 2, rooms[4][2]),
))

(16508, 43626)

## [Day 24: Arithmetic Logic Unit](https://adventofcode.com/2021/day/24)

In [44]:
input_24 = readlines("24.txt")
c5s = parse.(Int, last.(split.(input_24[5:18:end])))
c6s = parse.(Int, last.(split.(input_24[6:18:end])))
c16s = parse.(Int, last.(split.(input_24[16:18:end])))
cs = collect(zip(c5s, c6s, c16s))

function process_block((x, y, z), (w, c5, c6, c16))
    x = !(z % 26 + c6 == w)
    y = (w + c16) * x
    z = (25 * x + 1) * div(z, c5) + y
    (Int(x), y, z)
end

function process_all_blocks(ws, cs)
    x = y = z = d = 0
    for (w, (c5, c6, c16)) in zip(ws, cs)
        d -= (c5 == 26)
        print("    "^d, (w, z % 26 + c6, c5 == 1 ? c16 : c6), " ")
        (x, y, z) = process_block((x, y, z), (w, c5, c6, c16))
        println(digits(z; base = 26))
        d += (c5 == 1)
    end
    z
end

process_all_blocks (generic function with 1 method)

In [45]:
ws = [
    9,
        6,
            2,
                9,
                    9, 8,
                    9, 6,
                4,
                    4, 9,
            9,
        9,
    7
]
foreach(print, ws); println()
process_all_blocks(ws, cs)

96299896449997
(9, 10, 12) [21]
    (6, 33, 7) [13, 21]
        (2, 23, 8) [10, 13, 21]
            (9, 22, 8) [17, 10, 13, 21]
                (9, 28, 15) [24, 17, 10, 13, 21]
                (8, 8, -16) [17, 10, 13, 21]
                (9, 27, 8) [17, 17, 10, 13, 21]
                (6, 6, -11) [17, 10, 13, 21]
            (4, 4, -13) [10, 13, 21]
            (4, 23, 13) [17, 10, 13, 21]
            (9, 9, -8) [10, 13, 21]
        (9, 9, -1) [13, 21]
    (9, 9, -4) [21]
(7, 7, -14) [0]


0

In [46]:
ws = [
    3,
        1,
            1,
                6,
                    2, 1,
                    4, 1,
                1,
                    1, 6,
            8,
        4,
    1
]
foreach(print, ws); println()
process_all_blocks(ws, cs)

31162141116841
(3, 10, 12) [15]
    (1, 27, 7) [8, 15]
        (1, 18, 8) [9, 8, 15]
            (6, 21, 8) [14, 9, 8, 15]
                (2, 25, 15) [17, 14, 9, 8, 15]
                (1, 1, -16) [14, 9, 8, 15]
                (4, 24, 8) [12, 14, 9, 8, 15]
                (1, 1, -11) [14, 9, 8, 15]
            (1, 1, -13) [9, 8, 15]
            (1, 22, 13) [14, 9, 8, 15]
            (6, 6, -8) [9, 8, 15]
        (8, 8, -1) [8, 15]
    (4, 4, -4) [15]
(1, 1, -14) [0]


0

## [Day 25: Sea Cucumber](https://adventofcode.com/2021/day/25)

In [47]:
input_25 = readlines("25.txt")
seafloor = permutedims(reduce(hcat, collect.(input_25)))

function step_cucumbers(seafloor)
    seafloor_east = circshift(seafloor, (0, 1))
    seafloor_west = circshift(seafloor, (0, -1))
    seafloor =
        ifelse.(
            seafloor_east .== '>' .&& seafloor .== '.',
            seafloor_east,
            ifelse.(seafloor .== '>' .&& seafloor_west .== '.', '.', seafloor),
        )
    seafloor_south = circshift(seafloor, (1, 0))
    seafloor_north = circshift(seafloor, (-1, 0))
    ifelse.(
        seafloor_south .== 'v' .&& seafloor .== '.',
        seafloor_south,
        ifelse.(seafloor .== 'v' .&& seafloor_north .== '.', '.', seafloor),
    )
end

function stop_cucumbers(seafloor)
    for i in 1:typemax(Int)
        new_seafloor = step_cucumbers(seafloor)
        new_seafloor == seafloor && return i
        seafloor = new_seafloor
    end
end

stop_cucumbers(seafloor)

523